In [28]:
# import shapefile
import numpy as np
import geopandas as gpd
import pandas as pd
import ndjson
from shapely.geometry import Polygon, Point
import os, glob

In [2]:
# shapefile
sf = gpd.read_file("/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/sd2010zw_region/woonbrt10_region.shp")
sf1 = sf.to_crs(4326)
# sf1

## OSM

In [157]:
classes = ['gentrified', 'non-gentrified']
root = '/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/OSM'

dirs = [glob.glob(os.path.join(root, c)) for c in classes]
# for dir in dirs:
#     print(dir[0])
dirs[0][0] # gentrified
# dirs[1][0] # non-gentrified

'/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/OSM/gentrified'

In [321]:
root = '/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/OSM/gentrified'
folders = glob.glob(os.path.join(root, '*'))

for i,folder in enumerate(folders[0:5]):
    coord = os.path.split(folder)[1].split('-')[1].split('_')[3:5] # get image coordinates in folder name
    print(coord)
    # coord = Point([float(x) for x in coord]) # turn into points

    # print(i,'--', os.path.split(folder)[1])
    
    # for i,row in sf1.iterrows():    # sf1: Geopandas df with geometry transformed to lon-lat (EPSG:4326)
    #     if row['geometry'].contains(coord):
    #         nbhd = row['BCNAAM']
    #         print(coord, '--', nbhd)

['4.8272414366203', '52.3549889568876']
['4.82684351091589', '52.3551804351019']
['4.82677351379842', '52.3551702410545']
['4.82684351091589', '52.3551804351019']
['4.82714364531735', '52.3548132272462']


## StreetSwipe

In [80]:
with open('/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/StreetSwipe/database/post_202007/data-light-latest.ndjson') as f:
    file = ndjson.load(f)
df_post = pd.json_normalize(file, 'annotations', ['id']).drop(columns=['data.name','data.urls.square.jpg'])
df_post = df_post[['id','data.geometry.coordinates']]
df_post.rename(columns={'data.geometry.coordinates':'lon-lat'}, inplace = True)
df_post

,id,lon-lat
0,osm:node:1223236885,"[4.893005, 52.3799291]"
1,osm:node:1772991829,"[4.8895298, 52.368966]"
2,osm:node:1977672311,"[4.8026959, 52.3589167]"
3,osm:node:2507166317,"[4.8995483, 52.356797]"
4,osm:node:2724000369,"[4.8906526, 52.3689837]"
...,...,...
524,osm:node:1174575276,"[4.9004653, 52.3743281]"
525,osm:node:2423888952,"[4.8750168, 52.3635585]"
526,osm:node:1103394120,"[4.8888766, 52.3695703]"
527,osm:node:1223057252,"[4.8872544, 52.38277]"


In [81]:
gdf_post = gpd.GeoDataFrame(df_post,
                            geometry=gpd.points_from_xy(df_post['lon-lat'].str[0], df_post['lon-lat'].str[1]),
                            crs="EPSG:4326")
gdf_post

,id,lon-lat,geometry
0,osm:node:1223236885,"[4.893005, 52.3799291]",POINT (4.89300 52.37993)
1,osm:node:1772991829,"[4.8895298, 52.368966]",POINT (4.88953 52.36897)
2,osm:node:1977672311,"[4.8026959, 52.3589167]",POINT (4.80270 52.35892)
3,osm:node:2507166317,"[4.8995483, 52.356797]",POINT (4.89955 52.35680)
4,osm:node:2724000369,"[4.8906526, 52.3689837]",POINT (4.89065 52.36898)
...,...,...,...
524,osm:node:1174575276,"[4.9004653, 52.3743281]",POINT (4.90047 52.37433)
525,osm:node:2423888952,"[4.8750168, 52.3635585]",POINT (4.87502 52.36356)
526,osm:node:1103394120,"[4.8888766, 52.3695703]",POINT (4.88888 52.36957)
527,osm:node:1223057252,"[4.8872544, 52.38277]",POINT (4.88725 52.38277)


In [82]:
sf1['nbhd_geom'] = sf1.geometry
df_nbhd_post = gpd.sjoin(gdf_post, sf1, predicate='intersects', how='left')[['id','lon-lat','geometry','BCNAAM','nbhd_geom']]
df_nbhd_post.rename(columns={'BCNAAM':'neighborhood'}, inplace = True)
df_nbhd_post

,id,lon-lat,geometry,neighborhood,nbhd_geom
0,osm:node:1223236885,"[4.893005, 52.3799291]",POINT (4.89300 52.37993),Haarlemmerbuurt Oost,"POLYGON ((4.89396 52.37882, 4.89019 52.37965, ..."
1,osm:node:1772991829,"[4.8895298, 52.368966]",POINT (4.88953 52.36897),Begijnhofbuurt,"POLYGON ((4.89063 52.37261, 4.89122 52.37271, ..."
2,osm:node:1977672311,"[4.8026959, 52.3589167]",POINT (4.80270 52.35892),Osdorpplein e.o.,"POLYGON ((4.80079 52.35517, 4.79920 52.35799, ..."
3,osm:node:2507166317,"[4.8995483, 52.356797]",POINT (4.89955 52.35680),Sarphatiparkbuurt,"POLYGON ((4.89697 52.35689, 4.89848 52.35734, ..."
4,osm:node:2724000369,"[4.8906526, 52.3689837]",POINT (4.89065 52.36898),Begijnhofbuurt,"POLYGON ((4.89063 52.37261, 4.89122 52.37271, ..."
...,...,...,...,...,...
524,osm:node:1174575276,"[4.9004653, 52.3743281]",POINT (4.90047 52.37433),Burgwallen Oost,"POLYGON ((4.89574 52.37185, 4.89939 52.37473, ..."
525,osm:node:2423888952,"[4.8750168, 52.3635585]",POINT (4.87502 52.36356),Helmersbuurt Oost,"POLYGON ((4.87754 52.36678, 4.87888 52.36570, ..."
526,osm:node:1103394120,"[4.8888766, 52.3695703]",POINT (4.88888 52.36957),Spuistraat Zuid,"POLYGON ((4.88787 52.36876, 4.88840 52.37215, ..."
527,osm:node:1223057252,"[4.8872544, 52.38277]",POINT (4.88725 52.38277),Haarlemmerbuurt West,"POLYGON ((4.88419 52.38569, 4.89032 52.38254, ..."


In [83]:
scores_post = pd.read_csv("/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/StreetSwipe/database/post_202007/ss_afb_scores.csv", names = ['id','yes','no'] )
scores_post["label"] = np.where(scores_post['yes'] > scores_post['no'], "gentrified", "non-gentrified")
scores_post = scores_post.drop(columns=['yes', 'no'])
scores_post

,id,label
0,faillissementsdossier:24368214,non-gentrified
1,faillissementsdossier:33251073,non-gentrified
2,faillissementsdossier:60624485,non-gentrified
3,faillissementsdossier:63432250,non-gentrified
4,faillissementsdossier:63453428,gentrified
...,...,...
526,osm:node:7020259827,non-gentrified
527,osm:node:746910501,non-gentrified
528,osm:node:936580930,non-gentrified
529,osm:way:499929237,gentrified


In [84]:
df_nbhd_post = df_nbhd_post.merge(scores_post)[['id','label','lon-lat','geometry','neighborhood','nbhd_geom']]

In [86]:
df_nbhd_post['id'] = df_nbhd_post['id'].str.replace(':','-') + '.jpg'
df_nbhd_post

,id,label,lon-lat,geometry,neighborhood,nbhd_geom
0,osm-node-1223236885.jpg,gentrified,"[4.893005, 52.3799291]",POINT (4.89300 52.37993),Haarlemmerbuurt Oost,"POLYGON ((4.89396 52.37882, 4.89019 52.37965, ..."
1,osm-node-1772991829.jpg,non-gentrified,"[4.8895298, 52.368966]",POINT (4.88953 52.36897),Begijnhofbuurt,"POLYGON ((4.89063 52.37261, 4.89122 52.37271, ..."
2,osm-node-1977672311.jpg,non-gentrified,"[4.8026959, 52.3589167]",POINT (4.80270 52.35892),Osdorpplein e.o.,"POLYGON ((4.80079 52.35517, 4.79920 52.35799, ..."
3,osm-node-2507166317.jpg,non-gentrified,"[4.8995483, 52.356797]",POINT (4.89955 52.35680),Sarphatiparkbuurt,"POLYGON ((4.89697 52.35689, 4.89848 52.35734, ..."
4,osm-node-2724000369.jpg,non-gentrified,"[4.8906526, 52.3689837]",POINT (4.89065 52.36898),Begijnhofbuurt,"POLYGON ((4.89063 52.37261, 4.89122 52.37271, ..."
...,...,...,...,...,...,...
524,osm-node-1174575276.jpg,non-gentrified,"[4.9004653, 52.3743281]",POINT (4.90047 52.37433),Burgwallen Oost,"POLYGON ((4.89574 52.37185, 4.89939 52.37473, ..."
525,osm-node-2423888952.jpg,non-gentrified,"[4.8750168, 52.3635585]",POINT (4.87502 52.36356),Helmersbuurt Oost,"POLYGON ((4.87754 52.36678, 4.87888 52.36570, ..."
526,osm-node-1103394120.jpg,non-gentrified,"[4.8888766, 52.3695703]",POINT (4.88888 52.36957),Spuistraat Zuid,"POLYGON ((4.88787 52.36876, 4.88840 52.37215, ..."
527,osm-node-1223057252.jpg,non-gentrified,"[4.8872544, 52.38277]",POINT (4.88725 52.38277),Haarlemmerbuurt West,"POLYGON ((4.88419 52.38569, 4.89032 52.38254, ..."


In [382]:
# df_nbhd_post.to_csv('/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/StreetSwipe/database/post_202007/df_nbhd_post.csv')

In [87]:
with open('/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/StreetSwipe/database/post_202007/data-raw-latest.ndjson') as f:
    file = ndjson.load(f)
df = pd.json_normalize(file, 'annotations', ['id'])
df = df[['id','data.year']]
# df = df[df['data.year']<2015].reset_index(drop=True) # data before 2015 - to be excluded
# df['id'] = df['id'].str.replace(':','-') + '.jpg'
# df['data.year'] = df['data.year'].astype('int')
# df

,id,data.year
0,osm-node-2816318755.jpg,2009
1,osm-node-4688265763.jpg,2009
2,osm-node-2848302824.jpg,2008
3,osm-node-2848701163.jpg,2014
4,osm-node-2848301407.jpg,2008
...,...,...
84,osm-node-1310974815.jpg,2009
85,osm-node-2451055491.jpg,2010
86,osm-node-2754640987.jpg,2014
87,osm-node-1721802758.jpg,2014


# ---------------------------------------------------------

In [102]:
df_pre = pd.read_csv('/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/StreetSwipe/database/pre_202007/ss_afb_data.csv',
                     names=['id','year','city','street','lat','lon','img'])
df_pre = df_pre[['id','year','lat','lon']]
df_pre['lon'] = df_pre['lon'].replace("_", ".", regex=True)
df_pre['lat'] = df_pre['lat'].replace("_", ".", regex=True)
df_pre

,id,year,lat,lon
0,1,2009,52.350363,4.8572846
1,2,2009,52.3504455,4.8572465
2,3,2009,52.3504851,4.8572272
3,4,2009,52.3507459,4.8570525
4,5,2009,52.3508895,4.8569911
...,...,...,...,...
1907,1908,2019,52.3710722,4.8589981
1908,1909,2019,52.3711629,4.8589827
1909,1910,2013,NaN,NaN
1910,1911,2016,NaN,NaN


In [103]:
gdf_pre = gpd.GeoDataFrame(df_pre,
                           geometry=gpd.points_from_xy(df_pre['lon'], df_pre['lat']),
                           crs="EPSG:4326")
gdf_pre

,id,year,lat,lon,geometry
0,1,2009,52.350363,4.8572846,POINT (4.85728 52.35036)
1,2,2009,52.3504455,4.8572465,POINT (4.85725 52.35045)
2,3,2009,52.3504851,4.8572272,POINT (4.85723 52.35049)
3,4,2009,52.3507459,4.8570525,POINT (4.85705 52.35075)
4,5,2009,52.3508895,4.8569911,POINT (4.85699 52.35089)
...,...,...,...,...,...
1907,1908,2019,52.3710722,4.8589981,POINT (4.85900 52.37107)
1908,1909,2019,52.3711629,4.8589827,POINT (4.85898 52.37116)
1909,1910,2013,NaN,NaN,POINT EMPTY
1910,1911,2016,NaN,NaN,POINT EMPTY


In [104]:
df_nbhd_pre = gpd.sjoin(gdf_pre, sf1, predicate='intersects', how='left')[['id','year','lon','lat','geometry','BCNAAM','nbhd_geom']]
df_nbhd_pre.rename(columns={'BCNAAM':'neighborhood'}, inplace = True)
df_nbhd_pre['lon-lat'] = df_nbhd_pre[['lon', 'lat']].values.tolist()
df_nbhd_pre = df_nbhd_pre[['id','year', 'lon-lat', 'geometry', 'neighborhood', 'nbhd_geom']]
df_nbhd_pre

,id,year,lon-lat,geometry,neighborhood,nbhd_geom
0,1,2009,"[4.8572846, 52.350363]",POINT (4.85728 52.35036),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
1,2,2009,"[4.8572465, 52.3504455]",POINT (4.85725 52.35045),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
2,3,2009,"[4.8572272, 52.3504851]",POINT (4.85723 52.35049),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
3,4,2009,"[4.8570525, 52.3507459]",POINT (4.85705 52.35075),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
4,5,2009,"[4.8569911, 52.3508895]",POINT (4.85699 52.35089),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
...,...,...,...,...,...,...
1907,1908,2019,"[4.8589981, 52.3710722]",POINT (4.85900 52.37107),Kortenaerkwartier,"POLYGON ((4.85991 52.37149, 4.86341 52.37082, ..."
1908,1909,2019,"[4.8589827, 52.3711629]",POINT (4.85898 52.37116),Kortenaerkwartier,"POLYGON ((4.85991 52.37149, 4.86341 52.37082, ..."
1909,1910,2013,"[nan, nan]",POINT EMPTY,NaN,None
1910,1911,2016,"[nan, nan]",POINT EMPTY,NaN,None


In [105]:
scores_pre = pd.read_csv("/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/StreetSwipe/database/pre_202007/ss_afb_scores.csv", names = ['id','yes','no'])
scores_pre["label"] = np.where(scores_pre['yes'] > scores_pre['no'], "gentrified", "non-gentrified")
scores_pre = scores_pre.drop(columns=['yes', 'no'])
scores_pre

,id,label
0,1,non-gentrified
1,2,non-gentrified
2,3,non-gentrified
3,4,non-gentrified
4,5,non-gentrified
...,...,...
1907,1908,gentrified
1908,1909,gentrified
1909,1910,non-gentrified
1910,1911,non-gentrified


In [106]:
df_nbhd_pre = df_nbhd_pre.merge(scores_pre)[['id','label','year','lon-lat','geometry','neighborhood','nbhd_geom']]
df_nbhd_pre

,id,label,year,lon-lat,geometry,neighborhood,nbhd_geom
0,1,non-gentrified,2009,"[4.8572846, 52.350363]",POINT (4.85728 52.35036),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
1,2,non-gentrified,2009,"[4.8572465, 52.3504455]",POINT (4.85725 52.35045),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
2,3,non-gentrified,2009,"[4.8572272, 52.3504851]",POINT (4.85723 52.35049),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
3,4,non-gentrified,2009,"[4.8570525, 52.3507459]",POINT (4.85705 52.35075),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
4,5,non-gentrified,2009,"[4.8569911, 52.3508895]",POINT (4.85699 52.35089),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
...,...,...,...,...,...,...,...
1907,1908,gentrified,2019,"[4.8589981, 52.3710722]",POINT (4.85900 52.37107),Kortenaerkwartier,"POLYGON ((4.85991 52.37149, 4.86341 52.37082, ..."
1908,1909,gentrified,2019,"[4.8589827, 52.3711629]",POINT (4.85898 52.37116),Kortenaerkwartier,"POLYGON ((4.85991 52.37149, 4.86341 52.37082, ..."
1909,1910,non-gentrified,2013,"[nan, nan]",POINT EMPTY,NaN,None
1910,1911,non-gentrified,2016,"[nan, nan]",POINT EMPTY,NaN,None


In [107]:
df_nbhd_pre['id'] = df_nbhd_pre['id'].astype(str) + '.jpg'
df_nbhd_pre

,id,label,year,lon-lat,geometry,neighborhood,nbhd_geom
0,1.jpg,non-gentrified,2009,"[4.8572846, 52.350363]",POINT (4.85728 52.35036),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
1,2.jpg,non-gentrified,2009,"[4.8572465, 52.3504455]",POINT (4.85725 52.35045),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
2,3.jpg,non-gentrified,2009,"[4.8572272, 52.3504851]",POINT (4.85723 52.35049),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
3,4.jpg,non-gentrified,2009,"[4.8570525, 52.3507459]",POINT (4.85705 52.35075),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
4,5.jpg,non-gentrified,2009,"[4.8569911, 52.3508895]",POINT (4.85699 52.35089),Valeriusbuurt West,"POLYGON ((4.86351 52.35321, 4.86381 52.35281, ..."
...,...,...,...,...,...,...,...
1907,1908.jpg,gentrified,2019,"[4.8589981, 52.3710722]",POINT (4.85900 52.37107),Kortenaerkwartier,"POLYGON ((4.85991 52.37149, 4.86341 52.37082, ..."
1908,1909.jpg,gentrified,2019,"[4.8589827, 52.3711629]",POINT (4.85898 52.37116),Kortenaerkwartier,"POLYGON ((4.85991 52.37149, 4.86341 52.37082, ..."
1909,1910.jpg,non-gentrified,2013,"[nan, nan]",POINT EMPTY,NaN,None
1910,1911.jpg,non-gentrified,2016,"[nan, nan]",POINT EMPTY,NaN,None


In [391]:
# df_nbhd_pre.to_csv('/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/StreetSwipe/database/pre_202007/df_nbhd_pre.csv')

## AmsPano

In [260]:
root_d = '/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/AmsPano'
df_pano = pd.DataFrame(columns = ['id','label','lonlat','neighborhood'])

for c in glob.glob(os.path.join(root_d,"*")):
    label = os.path.split(c)[1]
    
    nbhd_dir = glob.glob(os.path.join(c, "*"))
    for n in nbhd_dir:
        neighborhood = os.path.split(n)[1]
        
        loc_dir = glob.glob(os.path.join(n, "*"))
        for l in loc_dir:          
            id_dir = glob.glob(os.path.join(l, "*[!.jpg]"))
            for i in id_dir:
                id = os.path.split(i)[1]
                lonlat = [os.path.split(i)[1].split('-')[1].split('_')[3:5]]
            
            dict = {'id':id, 'label':label, 'lonlat':lonlat, 'neighborhood':neighborhood}
            entry = pd.DataFrame.from_dict(dict)
            df_pano = pd.concat([df_pano, entry], ignore_index=True)
df_pano.rename(columns={'lonlat':'lon-lat'}, inplace=True)
df_pano

,id,label,lon-lat,neighborhood
0,20190513TMX7316010203-001178pano_0001_004104_4...,gentrified,"[4.87904431374453, 52.3719160051184]",Elandsgrachtbuurt
1,20210317TMX7316010203-002044pano_0000_000784_4...,gentrified,"[4.87976847022201, 52.3708683069787]",Elandsgrachtbuurt
2,20230220TMX7316010203-002978pano_0004_000038_4...,gentrified,"[4.88273090828026, 52.3733198656603]",Elandsgrachtbuurt
3,20220429TMX7316010203-002550pano_0000_000712_4...,gentrified,"[4.88010837247913, 52.3703502339621]",Elandsgrachtbuurt
4,20230116TMX7316010203-002939pano_0000_000094_4...,gentrified,"[4.88102143662789, 52.3727939863886]",Elandsgrachtbuurt
...,...,...,...,...
4167,20190611TMX7316010203-001213pano_0000_002817_4...,non-gentrified,"[4.98104983585991, 52.2950874855789]",Gaasperdam Zuid
4168,20200626TMX7316010203-001774pano_0000_001072_4...,non-gentrified,"[4.97729160064709, 52.2928014199608]",Gaasperdam Zuid
4169,20200131TMX7316010203-001615pano_0005_000115_4...,non-gentrified,"[4.97703853540859, 52.2932516046259]",Gaasperdam Zuid
4170,20220620TMX7316010203-002612pano_0002_000997_4...,non-gentrified,"[4.97800142839781, 52.2941163980353]",Gaasperdam Zuid


In [262]:
gdf_pano = gpd.GeoDataFrame(df_pano,
                           geometry=gpd.points_from_xy(df_pano['lon-lat'].str[0], df_pano['lon-lat'].str[1]),
                           crs="EPSG:4326")
gdf_pano

,id,label,lon-lat,neighborhood,geometry
0,20190513TMX7316010203-001178pano_0001_004104_4...,gentrified,"[4.87904431374453, 52.3719160051184]",Elandsgrachtbuurt,POINT (4.87904 52.37192)
1,20210317TMX7316010203-002044pano_0000_000784_4...,gentrified,"[4.87976847022201, 52.3708683069787]",Elandsgrachtbuurt,POINT (4.87977 52.37087)
2,20230220TMX7316010203-002978pano_0004_000038_4...,gentrified,"[4.88273090828026, 52.3733198656603]",Elandsgrachtbuurt,POINT (4.88273 52.37332)
3,20220429TMX7316010203-002550pano_0000_000712_4...,gentrified,"[4.88010837247913, 52.3703502339621]",Elandsgrachtbuurt,POINT (4.88011 52.37035)
4,20230116TMX7316010203-002939pano_0000_000094_4...,gentrified,"[4.88102143662789, 52.3727939863886]",Elandsgrachtbuurt,POINT (4.88102 52.37279)
...,...,...,...,...,...
4167,20190611TMX7316010203-001213pano_0000_002817_4...,non-gentrified,"[4.98104983585991, 52.2950874855789]",Gaasperdam Zuid,POINT (4.98105 52.29509)
4168,20200626TMX7316010203-001774pano_0000_001072_4...,non-gentrified,"[4.97729160064709, 52.2928014199608]",Gaasperdam Zuid,POINT (4.97729 52.29280)
4169,20200131TMX7316010203-001615pano_0005_000115_4...,non-gentrified,"[4.97703853540859, 52.2932516046259]",Gaasperdam Zuid,POINT (4.97704 52.29325)
4170,20220620TMX7316010203-002612pano_0002_000997_4...,non-gentrified,"[4.97800142839781, 52.2941163980353]",Gaasperdam Zuid,POINT (4.97800 52.29412)


In [267]:
df_nbhd_pano = gpd.sjoin(gdf_pano, sf1, predicate='intersects', how='left')[['id','label','lon-lat','geometry','BCNAAM','nbhd_geom']]
df_nbhd_pano.rename(columns={'BCNAAM':'neighborhood'}, inplace = True)
df_nbhd_pano

,id,label,lon-lat,geometry,neighborhood,nbhd_geom
0,20190513TMX7316010203-001178pano_0001_004104_4...,gentrified,"[4.87904431374453, 52.3719160051184]",POINT (4.87904 52.37192),Elandsgrachtbuurt,"POLYGON ((4.88259 52.36892, 4.87908 52.36774, ..."
1,20210317TMX7316010203-002044pano_0000_000784_4...,gentrified,"[4.87976847022201, 52.3708683069787]",POINT (4.87977 52.37087),Elandsgrachtbuurt,"POLYGON ((4.88259 52.36892, 4.87908 52.36774, ..."
2,20230220TMX7316010203-002978pano_0004_000038_4...,gentrified,"[4.88273090828026, 52.3733198656603]",POINT (4.88273 52.37332),Elandsgrachtbuurt,"POLYGON ((4.88259 52.36892, 4.87908 52.36774, ..."
3,20220429TMX7316010203-002550pano_0000_000712_4...,gentrified,"[4.88010837247913, 52.3703502339621]",POINT (4.88011 52.37035),Elandsgrachtbuurt,"POLYGON ((4.88259 52.36892, 4.87908 52.36774, ..."
4,20230116TMX7316010203-002939pano_0000_000094_4...,gentrified,"[4.88102143662789, 52.3727939863886]",POINT (4.88102 52.37279),Elandsgrachtbuurt,"POLYGON ((4.88259 52.36892, 4.87908 52.36774, ..."
...,...,...,...,...,...,...
4167,20190611TMX7316010203-001213pano_0000_002817_4...,non-gentrified,"[4.98104983585991, 52.2950874855789]",POINT (4.98105 52.29509),Gaasperdam Zuid,"POLYGON ((4.97416 52.29555, 4.97596 52.29585, ..."
4168,20200626TMX7316010203-001774pano_0000_001072_4...,non-gentrified,"[4.97729160064709, 52.2928014199608]",POINT (4.97729 52.29280),Reigersbos Zuid,"POLYGON ((4.96731 52.29119, 4.96772 52.29132, ..."
4169,20200131TMX7316010203-001615pano_0005_000115_4...,non-gentrified,"[4.97703853540859, 52.2932516046259]",POINT (4.97704 52.29325),Gaasperdam Zuid,"POLYGON ((4.97416 52.29555, 4.97596 52.29585, ..."
4170,20220620TMX7316010203-002612pano_0002_000997_4...,non-gentrified,"[4.97800142839781, 52.2941163980353]",POINT (4.97800 52.29412),Gaasperdam Zuid,"POLYGON ((4.97416 52.29555, 4.97596 52.29585, ..."


In [272]:
df_AmsPano.to_csv('/Users/vanh/Documents/Master/Thesis/w.nosync/MSc-Thesis-Gentrification-and-storefront-signage/Data/AmsPano/df_AmsPano.csv')